<a href="https://colab.research.google.com/github/rikideguchi/Signate-HiroshimaQuest/blob/master/%E3%81%B2%E3%82%8D%E3%81%97%E3%81%BEQuest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv('../train_pitch.csv')
test = pd.read_csv('../test_pitch.csv')
data = pd.concat([train, test], sort = False)

In [0]:
data['打者守備位置'].replace(['投', '捕', '一', '二', '三', '遊', '左', '中', '右'], [0, 1, 1, 1, 1, 1, 1, 1, 1], inplace = True)

data['プレイ前走者状況'].replace(['___', '_2_', '1__', '12_', '_23', '1_3', '__3', '123'], 
                         ['0', '1', '2', '3', '4', '5', '6', '7'], inplace = True)

data['一塁走者ID'].fillna(0, inplace = True)

data['プレイ前走者状況'] = data['プレイ前走者状況'].astype(np.int8)
data['データ内連番'] = data['データ内連番'].astype(np.int32)
data['球種'] = data['球種'].astype(np.float16)
data['イニング'] = data['イニング'].astype(np.int8)
data['投手ID'] = data['投手ID'].astype(np.int32)
data['打者ID'] = data['打者ID'].astype(np.int32)
data['プレイ前ボール数'] = data['プレイ前ボール数'].astype(np.int8)
data['プレイ前ストライク数'] = data['プレイ前ストライク数'].astype(np.int8)
data['プレイ前走者状況'] = data['プレイ前走者状況'].astype(np.int8)
data['一塁走者ID'] = data['一塁走者ID'].astype(np.float32)
data['捕手ID'] = data['捕手ID'].astype(np.int32)

In [0]:
data.drop(['年度','試合種別詳細', '表裏','日付', '時刻', '投手登板順', '投手試合内対戦打者数', '打者チームID', '打者試合内打席数', '球場名',
           'プレイ前ホームチーム得点数','投手チームID','投手試合内投球数', 'プレイ前アウェイチーム得点数','プレイ前アウト数', '打者守備位置', 
           '一塁手ID', '二塁手ID','三塁手ID', '遊撃手ID', '左翼手ID', '中堅手ID', '右翼手ID','投手役割', '投手投球左右', '試合ID',
           'ホームチームID', 'アウェイチームID', '成績対象投手ID', '三塁走者ID', '打者打順', '二塁走者ID', 'イニング内打席数', 
           '試合内連番', '打者打席左右', '投手イニング内投球数', '打席内投球数', '投球位置区域', '試合内投球数', '球場ID', '成績対象打者ID'], axis = 1, inplace = True)

In [0]:
data.rename(columns = {'データ内連番': 'ID', '球種':'Ball type',
                       'イニング':'Inning',
                       '投手ID':'Pitcher ID','投手役割':'Pitcher role', 
                       '投手投球左右':'L/R Pitcher','打者ID':'batter ID',
                       'プレイ前ホームチーム得点数':'The sum of points',
                       'プレイ前ボール数':'The amount of balls', 'プレイ前ストライク数':'the amount of strike', 
                       'プレイ前走者状況':'The runner situation','捕手ID':'catcher id', '一塁走者ID':'First runner'}, inplace = True)

In [0]:
train = data[:len(train)]
test = data[len(train):]

In [0]:
y_train = train['Ball type']
X_train = train.drop('Ball type', axis = 1)
X_test = test.drop('Ball type', axis = 1)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size = 0.4, 
                    random_state = 0, stratify = y_train)

In [0]:
categorical_features = ['Inning', 'Pitcher ID',
       'batter ID','The amount of balls', 'the amount of strike',
       'The runner situation', 'First runner',
       'catcher id']

In [0]:
import lightgbm as lgb
import optuna
from sklearn.metrics import log_loss

def objective(trial):
    params = {
        'objective':'multiclass', 
        'max_bin': trial.suggest_int('max_bin', 255, 500), 
        'learning_rate':0.05, 
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
        'num_class': 8
    }
    
    lgb_train = lgb.Dataset(X_train, y_train, 
                           categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train,  
                          categorical_feature=categorical_features)
    
    model = lgb.train(params, lgb_train, 
                     valid_sets = [lgb_train, lgb_eval], 
                     verbose_eval = 50, 
                     num_boost_round = 200, 
                     early_stopping_rounds = 10)
    
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [0]:
study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed = 0))
study.optimize(objective, n_trials = 40)
study.best_params

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning:

Using categorical_feature in Dataset.

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning:

categorical_feature in param dict is overridden.



Training until validation scores don't improve for 10 rounds.
[50]	training's multi_logloss: 1.11935	valid_1's multi_logloss: 1.23302
[100]	training's multi_logloss: 0.980298	valid_1's multi_logloss: 1.18336
[150]	training's multi_logloss: 0.901505	valid_1's multi_logloss: 1.17585
Early stopping, best iteration is:
[152]	training's multi_logloss: 0.899254	valid_1's multi_logloss: 1.17584
